# Sentiment Analysis with an Recurrent Neural Networks (RNN)

In [9]:
import pandas as pd
import numpy as np
import re  
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding


In [10]:
data = pd.read_csv('swiggy.csv')
print("Columns in the dataset:")
print(data.columns.tolist())


Columns in the dataset:
['ID', 'Area', 'City', 'Restaurant Price', 'Avg Rating', 'Total Rating', 'Food Item', 'Food Type', 'Delivery Time', 'Review']


In [11]:
data["Review"] = data["Review"].str.lower()
data["Review"] = data["Review"].replace(r'[^a-z0-9\s]', '', regex=True)

data['sentiment'] = data['Avg Rating'].apply(lambda x: 1 if x > 3.5 else 0)
data = data.dropna() 


In [12]:
max_features = 5000  
max_length = 200    

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data["Review"])
X = pad_sequences(tokenizer.texts_to_sequences(data["Review"]), maxlen=max_length)
y = data['sentiment'].values  


In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)


In [14]:
model = Sequential([
    Embedding(input_dim=max_features, output_dim=16, input_length=max_length),
    SimpleRNN(64, activation='tanh', return_sequences=False),                   
    Dense(1, activation='sigmoid')                                          
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


c:\Users\it\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

score = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {score[1]:.2f}")


Epoch 1/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 12s 42ms/step - accuracy: 0.6975 - loss: 0.6130 - val_accuracy: 0.7156 - val_loss: 0.5990
Epoch 2/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.7166 - loss: 0.5978 - val_accuracy: 0.7156 - val_loss: 0.5972
Epoch 3/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.7136 - loss: 0.5983 - val_accuracy: 0.7156 - val_loss: 0.6022
Epoch 4/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.7196 - loss: 0.5939 - val_accuracy: 0.7156 - val_loss: 0.5979
Epoch 5/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.7039 - loss: 0.6059 - val_accuracy: 0.7156 - val_loss: 0.6010
Test accuracy: 0.72


In [16]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"
    
sample_review = "The food was great."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")


Review: The food was great.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step
Sentiment: Positive (Probability: 0.75)


# Sentiment Analysis using VADER

In [1]:
# import SentimentIntensityAnalyzer class from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Function to print sentiments of the sentence.
def sentiment_scores(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    # polarity_scores method of SentimentIntensityAnalyzer object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    print("Overall sentiment dictionary is : ", sentiment_dict)
    print("Sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    print("Sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    print("Sentence was rated as ", sentiment_dict['pos']*100, "% Positive")

    print("Sentence Overall Rated As", end=" ")

    # Decide sentiment as positive, negative, or neutral
    if sentiment_dict['compound'] >= 0.05 :
        print("Positive")
    elif sentiment_dict['compound'] <= -0.05 :
        print("Negative")
    else :
        print("Neutral")

# Driver code to test the function
if __name__ == "__main__" :

    print("\n1st Statement:")
    sentence = "Geeks For Geeks is the best portal for computer science engineering students."
    sentiment_scores(sentence)

    print("\n2nd Statement:")
    sentence = "Study is going on as usual."
    sentiment_scores(sentence)

    print("\n3rd Statement:")
    sentence = "I am very sad today."
    sentiment_scores(sentence)


1st Statement:
Overall sentiment dictionary is :  {'neg': 0.175, 'neu': 0.562, 'pos': 0.263, 'compound': 0.5267}
Sentence was rated as  17.5 % Negative
Sentence was rated as  56.2 % Neutral
Sentence was rated as  26.3 % Positive
Sentence Overall Rated As Positive

2nd Statement:
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentence was rated as  0.0 % Negative
Sentence was rated as  100.0 % Neutral
Sentence was rated as  0.0 % Positive
Sentence Overall Rated As Neutral

3rd Statement:
Overall sentiment dictionary is :  {'neg': 0.459, 'neu': 0.541, 'pos': 0.0, 'compound': -0.5256}
Sentence was rated as  45.9 % Negative
Sentence was rated as  54.1 % Neutral
Sentence was rated as  0.0 % Positive
Sentence Overall Rated As Negative
